# CONSUMERS AND PRODUCERS IN MONOPOLISTIC COMPETITION

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [1]:
import numpy as np
from scipy import optimize
import sympy as sm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

# Model description

**Write out the model in equations here.** 

Make sure you explain well the purpose of the model and comment so that other students who may not have seen it before can follow.  

In a monopolistic market with population of consumers and producers, an agent i maximizes utility as such: 

$U_i = C_i - \frac{1}{\lambda} L^{\lambda}_i$

Where the constraint is: 

$C_i = \frac{P_i}{P} Y_i$

$C_i$ is consumption, $L_i$ is the Labour supply, $Y_i$ is production of good i, $P_i$ is the price of good i, $P$ is aggregate price. The production is:

$Y_i = L^{\alpha}_i$

The demand is:

$Y_i = (\frac{P_i}{P})^{-\eta}$

In the above, Y is output and $\eta$ denotes elasticity of substitution

$\lambda > 1$, $\alpha \in (0,1)$, and $\eta > 1$, $x \equiv log X$. Furthermore, the aggregate demand is (with M being money supply):

$Y = \frac{M}{P}$

## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 